<a href="https://colab.research.google.com/github/AdamFulton/thesis/blob/main/time_series_1Dcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
import sklearn as sk
import sklearn.metrics.pairwise
from PIL import Image
import torch.nn.functional as f
from torch.utils.data.dataloader import DataLoader

%matplotlib inline

In [ ]:
sns.get_dataset_names()


['anagrams',
 'anscombe',
 'attention',
 'brain_networks',
 'car_crashes',
 'diamonds',
 'dots',
 'exercise',
 'flights',
 'fmri',
 'gammas',
 'geyser',
 'iris',
 'mpg',
 'penguins',
 'planets',
 'taxis',
 'tips',
 'titanic']

In [ ]:
flight_data = sns.load_dataset("flights")
flight_data.head()

years = []

for i in flight_data['year']:

    if i in years:
      continue
    
    else:
      years.append(i)

dataset = np.random.randint(1,1000, size=(1000,12))

count = 0
count2 = 0

sums = np.zeros(1000,float)


for i in range(0,1000):

  for j in range(0,12):


    sums[i] = sums[i] + dataset[i][j]


print(dataset[1].sum())
print(sums[1])


6599
6599.0


In [ ]:
from sklearn.preprocessing import MinMaxScaler

arr = np.array(sums,float)

arr = arr.reshape(-1,1)
scaler = MinMaxScaler(feature_range=(0,1))
dataset_normalized = scaler.fit_transform(dataset)

sums_normalized = scaler.fit_transform(arr)

In [ ]:
t = torch.tensor(dataset_normalized)
s = torch.tensor(sums_normalized)

final = []
for i in range(0,len(dataset_normalized)):

    temp = t[i].reshape(12,1)
    final.append((temp,s[i]))


In [ ]:
batch_size = 100

train_dl = DataLoader(final,batch_size)

In [ ]:
class CNN(nn.Module):
    """Model for human-activity-recognition."""
    def __init__(self, input_size, num_classes):
        super().__init__()

       
        self.features = nn.Sequential(
            nn.Conv1d(input_size, 64, 1),
            nn.MaxPool1d(1),
            nn.ReLU(),
            nn.Dropout(),
            nn.Conv1d(64, 64, 1),
             nn.MaxPool1d(1),
            nn.ReLU(),
            nn.Dropout(),
            nn.Conv1d(64, 64, 1),
            nn.MaxPool1d(1),
            nn.ReLU(),
            )
     
        self.classifier = nn.Sequential(
        	nn.Dropout(),
        	nn.Linear(6400,3200),
        	nn.ReLU(),
        	nn.Dropout(),
          nn.Linear(3200,100),
        	nn.ReLU(),
        	nn.Dropout(),
        	nn.Linear(100,1),
        	)

    def forward(self, x):
      x = self.features(x)
      x = torch.flatten(x)
     
      out = self.classifier(x)

      return out

In [ ]:
def get_default_device():

  if torch.cuda.is_available():
    return torch.device("cuda")

  else:
    return torch.device("cpu")

In [ ]:
device = get_default_device();
device

device(type='cuda')

In [ ]:
def to_device(data,device):

  if isinstance(data,(list,tuple)):
    return [to_device(x,device) for x in data]

  return data.to(device,non_blocking=True)

In [ ]:
model = CNN(12,12)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
epochs = 100

for i in range(epochs):
    correct = 0
    for seq, labels in train_dl:
        optimizer.zero_grad()

       
        seq = seq.to(torch.float)
        labels = labels.to(torch.float)
       
        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        
        optimizer.step()

    
    
    print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')
    
  

       

        



/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([100, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch:   0 loss: 0.02255758
epoch:   1 loss: 0.05320810
epoch:   2 loss: 0.02323345
epoch:   3 loss: 0.05019409
epoch:   4 loss: 0.17468005
epoch:   5 loss: 0.05693290
epoch:   6 loss: 0.11952846
epoch:   7 loss: 0.02241827
epoch:   8 loss: 0.04570170
epoch:   9 loss: 0.04513993
epoch:  10 loss: 0.01939293


KeyboardInterrupt: ignored